# Prompting Techniques and Interaction Patterns

## Learning Goals
- Explore basic prompting techniques: **zero-shot**, **few-shot**, and **chain-of-thought (CoT)**.
- Understand how prompting guides reasoning depth and style.
- Implement **interaction patterns**: ReAct and Plan-and-Act.
- Connect these methods to the design of LLM-based agents.

This notebook corresponds to Section *1.4 From Prompting Techniques to Interaction Patterns* in the lecture notes.

In [1]:
# %load get_llm.py
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama

# Load environment variables from .env
load_dotenv()

def get_llm(provider: str = "openai"):
    """
    Return a language model instance configured for either OpenAI or Ollama.

    This function centralizes the initialization of chat-based LLMs so that 
    notebooks and applications can switch seamlessly between cloud-based models 
    (OpenAI) and local models (Ollama).

    Parameters
    ----------
    provider : str, optional
        The backend provider to use. Options are:
        - "openai": returns a ChatOpenAI instance (requires OPENAI_API_KEY in .env).
        - "ollama": returns a ChatOllama instance (requires Ollama installed locally).
        Default is "openai".

    Returns
    -------
    langchain.chat_models.base.BaseChatModel
        A chat model instance that can be invoked with messages.

    Examples
    --------
    Initialize an OpenAI model (requires API key):

    >>> llm = get_llm("openai")
    >>> llm.invoke("Hello, how are you?")

    Initialize a local Ollama model (e.g., Gemma2 2B):

    >>> llm = get_llm("ollama")
    >>> llm.invoke("Summarize the benefits of reinforcement learning.")
    """
    if provider == "openai":
        return ChatOpenAI(
            model="gpt-4o-mini",  # can also be "gpt-4.1" or "gpt-4o"
            temperature=0
        )
    elif provider == "ollama":
        return ChatOllama(
            model="gemma2:2b",   # replace with any local model installed in Ollama
            temperature=0
        )
    else:
        raise ValueError("Unsupported provider. Use 'openai' or 'ollama'.")


In [2]:
# Setup
from langchain.schema import HumanMessage, SystemMessage
from dotenv import load_dotenv
load_dotenv()

llm = get_llm("openai")  # or get_llm("ollama")

## Zero-shot prompting
The model is asked to perform a task **without examples**.

In [3]:
question = "What is the capital of Brazil?"
response = llm.invoke([HumanMessage(content=question)])
print(response.content)

The capital of Brazil is Brasília.


## Few-shot prompting
The model is given **examples** to guide its output format and style.

In [4]:
examples = """
Q: What is the capital of France?
A: Paris

Q: What is the capital of Japan?
A: Tokyo
"""

prompt = f"""{examples}
Q: What is the capital of Brazil?
A:"""

response = llm.invoke([HumanMessage(content=prompt)])
print(response.content)

Brasília


## Chain-of-Thought (CoT)
The model is instructed to **reason step by step** before answering.

In [5]:
cot_prompt = "What is 15 multiplied by 3? Let's think step by step."
response = llm.invoke([HumanMessage(content=cot_prompt)])
print(response.content)

To find the product of 15 and 3, we can break it down step by step:

1. **Understand multiplication**: Multiplication is essentially repeated addition. So, multiplying 15 by 3 means we are adding 15 three times.

2. **Set up the addition**: 
   - 15 + 15 + 15

3. **Calculate the first two 15s**:
   - 15 + 15 = 30

4. **Add the third 15**:
   - 30 + 15 = 45

So, 15 multiplied by 3 equals 45. 

Therefore, \( 15 \times 3 = 45 \).


## ReAct: Reason + Act
The model alternates between **thinking** and **acting** by calling tools.

We simulate this with two simple tools:
- Calculator
- Dictionary

In [6]:
tools_description = """
You can use these tools:
- Calculator: perform math (e.g., {"tool": "calculator", "input": "2+2"})
- Dictionary: define words (e.g., {"tool": "dictionary", "input": "agent"})
Always output valid JSON when using a tool.
"""

react_prompt = f"""{tools_description}

Question: What is 12 * 7?
"""

response = llm.invoke([
    SystemMessage(content="You are an AI agent that reasons step by step."),
    HumanMessage(content=react_prompt)
])

print(response.content)

{"tool": "calculator", "input": "12*7"}


## Plan-and-Act
This pattern separates **planning** from **execution**.
- First, the model produces a plan in natural language.
- Then, the agent executes each step.

In [7]:
plan_prompt = """
You are a planning assistant.
Question: What steps are needed to find the square root of 144 and define the word 'agent'?
Respond with a step-by-step plan in numbered steps.
"""

plan = llm.invoke([HumanMessage(content=plan_prompt)])
print("Plan:\n", plan.content)

Plan:
 ### Steps to Find the Square Root of 144

1. **Understand the Concept**: Recognize that the square root of a number is a value that, when multiplied by itself, gives the original number.

2. **Identify the Number**: In this case, the number is 144.

3. **Use a Calculator (if needed)**: If you have access to a calculator, you can simply input 144 and use the square root function.

4. **Estimate the Square Root**: If you don't have a calculator, think of perfect squares close to 144. For example, 12 x 12 = 144.

5. **Confirm the Result**: Verify that 12 multiplied by itself equals 144.

6. **State the Answer**: Conclude that the square root of 144 is 12.

### Steps to Define the Word 'Agent'

1. **Identify the Context**: Determine the context in which the word 'agent' is being used (e.g., business, science, law, etc.).

2. **Research the Definition**: Look up the definition in a reliable dictionary or resource. 

3. **Consider Different Meanings**: Note that 'agent' can have multi

### Reflection
- Zero-shot and few-shot prompting demonstrate the importance of examples.
- Chain-of-thought prompting elicits **explicit reasoning**.
- ReAct and Plan-and-Act are **interaction patterns** that structure reasoning with actions.
- These patterns anticipate how modern frameworks like LangGraph organize multi-step workflows.

## Exercises
1. Modify the few-shot example to include **wrong examples**. What happens?
2. Extend the ReAct loop to include an additional tool, such as a **translator**.
3. In the Plan-and-Act section, implement execution of each step (calculator + dictionary).